In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: dataset_with_topics.csv  
  inflating: relationships.csv       
  inflating: topic_info_150.csv      
  inflating: us_senators.csv         


In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv('dataset_with_topics.csv')
senators = pd.read_csv('us_senators.csv')[['Senator', 'Party', 'screen_name', 'followers_count']]
senators.loc[senators["Party"].isin(['Independent[d]','Independent[n]', 'Democratic (DFL)[p]']), "Party"] = "Democratic"

In [ ]:
df.drop(['topic_1500', 'topic_500', 'topic_150'], axis=1, inplace=True)

In [ ]:
df = pd.merge(df, senators, left_on='username', right_on='screen_name')

In [ ]:
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

df = df[df['date'] >= '2020-08-01']
df = df[df['date'] <= '2023-02-01']


# Extract the month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
# Group the elements by 'Party', 'year', and 'month', summing 'retweets' and 'likes', and counting the total number of items
grouped_df = df.groupby(['Party', 'year', 'month']).agg({'retweets': 'sum', 'likes': 'sum', 'text': 'count'}).reset_index()

In [ ]:
# Create a combined column for year and month
grouped_df['year_month'] = grouped_df['year'].astype(str) + '-' + grouped_df['month'].astype(str).str.zfill(2)

grouped_df['popularity'] = (grouped_df.retweets + grouped_df.likes) / grouped_df.text

# Sort the DataFrame by the year and month column
grouped_df = grouped_df.sort_values('year_month')

# Plot the popularity trends for each party using Plotly
fig = px.line(grouped_df, x='year_month', y='popularity', color='Party', color_discrete_sequence=['blue', 'red'])
fig.update_xaxes(type='category')  # Ensure correct ordering of x-axis labels
fig.update_xaxes(title='Date')
fig.update_layout(height=400)
fig.show()

In [ ]:
# Group the elements by 'Party', 'year', and 'month', summing 'retweets' and 'likes', and counting the total number of items
grouped_df = df.groupby(['Senator', 'year', 'month']).agg({'retweets': 'sum', 'likes': 'sum', 'text': 'count', 'followers_count':'max'}).reset_index()

grouped_df = grouped_df[grouped_df.Senator.isin(['Rand Paul', 'Jon Ossoff'])]


grouped_df['popularity'] = (grouped_df.retweets + grouped_df.likes) / grouped_df.text
grouped_df['popularity'] = grouped_df['popularity']/grouped_df['popularity'].max()

# Create a combined column for year and month
grouped_df['year_month'] = grouped_df['year'].astype(str) + '-' + grouped_df['month'].astype(str).str.zfill(2)

# Sort the DataFrame by the year and month column
grouped_df = grouped_df.sort_values('year_month')

# Plot the popularity trends for each party using Plotly
fig = px.line(grouped_df, x='year_month', y='popularity', color='Senator', color_discrete_sequence=['blue', 'red'])
fig.update_xaxes(type='category')  # Ensure correct ordering of x-axis labels
fig.update_xaxes(title='Date')
fig.update_layout(height=400)
fig.show()

In [ ]:
grouped_df = df.groupby(['Senator']).agg({'retweets': 'sum', 'likes': 'sum', 'text': 'count', 'followers_count':'max'}).reset_index()
grouped_df['popularity'] = (grouped_df.retweets + grouped_df.likes + 0.25 * grouped_df.followers_count) / grouped_df.text
grouped_df['popularity'] = grouped_df['popularity']/grouped_df['popularity'].max()

In [ ]:
grouped_df[grouped_df.popularity >= 0.5]

,Senator,retweets,likes,text,followers_count,popularity
5,Bernie Sanders,4920872,29848230,1695,12496635,0.891287
45,Jon Ossoff,3582329,25171621,1804,1213409,0.642161
70,Mitt Romney,390254,3360570,170,2052952,1.000000


In [ ]:
grouped_df[grouped_df.Senator == 'Tommy Tuberville']

,Senator,retweets,likes,text,followers_count,popularity
97,Tommy Tuberville,32315,181786,587,77306,0.013561


In [ ]:
x = df.loc[(df.Senator == 'Jon Ossoff') & (df.month == 1) & (df.year == 2021)]
print(x[x.retweets == x.retweets.max()].text.item())
x.sort_values(by='retweets', ascending=False).text.head(5).iloc[-1]

POLLS ARE OPEN. VOTE.


'Today’s insurrectionist attack on the U.S. Capitol was incited by Trump’s poisonous lies &amp; flagrant assault on our Constitution. \n\nThe GOP must discard and disavow Trump once and for all, end its attacks on the electoral process, &amp; commit fully to the peaceful transfer of power.'